# django-rest-framework 구성하기


### 1. django-rest-framework를 설치

```
pip install djangorestframework
```

### 2. settings.py 수정

```python
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    
    'post',
    'rest_framework', #추가
    
]
...
#이하 추가
REST_FRAMEWORK = {
    'DEFAULT_PERMISSION_CLASSES': [
        'rest_framework.permissions.AllowAny',
    ]
}
```

### 3. 반환값을 JSON으로 직렬화(Serialize)
* DRF는 자동으로 pk값을 index로 지정해서 뿌려준다.
* 이제 api app을 추가하였으니 api루트로 들어왔을 때 데이터를 보낼 수 있게 해야한다.
* 어떤 형식의 데이터를 보낼지는 post app디렉토리 안의 view.py에서 정해진다.
* 중요한 것은 **serializers**이다.


> django + react 앱은 api요청을 통해 데이터를 주고 받는데 많은 사람들이 아다시피 api요청 및 반환값은 데이터포멧이 JSON(JavaScript Object Notation)으로 되어있다. 그래서 반환값을 JSON으로 직렬화(Serialize)해주는 것이 필요하다. 이때 필요한 것이 DRF의 serializers이다. 만들어져있는 파일이 아니니 직접 만들어 작성해야한다.

* safehome/serializers.py

```python
from rest_framework import serializers
from .models import Crime

class CrimeSerializer(serializers.ModelSerializer):
    class Meta:
        fields = (
            'id',
            'area',
            'murder',
            'robber',
            'rape',
            'theft',
            'violence',
            'total',
            'arr_total',
            'arrest'
        )
        model = Crime
```

* seializer가 작성되었으면 views.py를 작성

```python
#/safehome/views.py
from django.shortcuts import render
from rest_framework import generics

from .models import Crime
from .serializers import CrimeSerializer

class ListCrime(generics.ListCreateAPIView):
    queryset = Crime.objects.all()
    serializer_class = CrimeSerializer

class DetailCrime(generics.RetrieveUpdateDestroyAPIView):
    queryset = Crime.objects.all()
    serializer_class = CrimeSerializer
# Create your views here.

```

* api요청이 들어오면 Post 데이터를 보내야하므로 urls.py를 만들어서 작성

```python
from django.urls import path

from . import views

urlpatterns = [
    path('', views.ListCrime.as_view()),
    path('<int:pk>/', views.DetailCrime.as_view()),
]
```

* 이제 django api서버의 준비는 완료

* 0.0.0.0/api 로 접속해보면  List Crime을 확인해 볼 수 있다
![list_cirme](img/list_crime.PNG)

* script태그 안에서의 api를 통한 데이터의 접근제어를 위해 HTTP 접근제어 규약(CORS : Cross-Origin Resource Sharing)을 추가
* 기존의 HTTP요청에서는 img나 link태그 등으로 다른 호스트의 css나 이미지파일 등의 리소스를 가져오는 것이 가능한데 script태그로 쌓여진 코드에서의 다른 도메인에 대한 요청은 Same-origin policy에 대한 규약으로 인해 접근이 불가
*  react는 거의가 script요청에 의해 페이지를 그리는 방식이므로 이제 대한 제약 해제가 필요
* django로 돌아가고 있는 api서버와 페이지를 그려주는 react서버는 명목상 아예 다른 서버로 구분되기 때문이다.

* 패키지 설치
```
pip install django-cors-headers
```

* test_django/settings.py

```python
INSTALLED_APPS = [
    'corsheaders', # 추가
]

...

MIDDLEWARE = [
    'corsheaders.middleware.CorsMiddleware',     # 추가
    'django.middleware.common.CommonMiddleware', # 추가
]

...

#script안에서의 리소스 요청을 허용할 도메인 추가
# CORS_ORIGIN_WHITELIST = (
#     'localhost:3000/'
# )

# 모든 도메인 허용
CORS_ORIGIN_ALLOW_ALL = True

```


* 이제 react앱에서 django에서 전송해주는 데이터를 받아올 수 있다